In [2]:
from pathlib import Path
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

DATASET = Path("datasets/lotto/data_processed.csv")

# The Answer to the Ultimate Question of Life, the Universe, and Everything.
np.random.seed(42)

In [3]:
# Read into data frame
dataset = pd.read_csv(DATASET, header=0, sep=',', quotechar='"', parse_dates=['DrawDate'], dtype={'PrizeType': str})

In [18]:
df = dataset.copy()
len(df.columns)

115

In [19]:
for ind, column in enumerate(df.columns):
    print('"{}"'.format(column), end=', ')
    if ind % 5 == 0:
        print()

"DrawNo", 
"DrawDate", "PrizeType", "LuckyNo", "DaysSinceLastExactMatch", "DaysUntilNextExactMatch", 
"DaysSinceLastAnyMatch", "DaysUntilNextAnyMatch", "1st_digit", "2nd_digit", "3rd_digit", 
"4th_digit", "DrawYear", "DrawMonth", "DrawWeek", "DrawDay", 
"DrawDayofweek", "DrawDayofyear", "CumProbaExactMatch", "CumProbaAnyMatch", "0_1", 
"0_2", "0_3", "0_4", "0_5", "0_6", 
"0_7", "0_8", "0_9", "1_2", "1_3", 
"1_4", "1_5", "1_6", "1_7", "1_8", 
"1_9", "2_3", "2_4", "2_5", "2_6", 
"2_7", "2_8", "2_9", "3_4", "3_5", 
"3_6", "3_7", "3_8", "3_9", "4_5", 
"4_6", "4_7", "4_8", "4_9", "5_6", 
"5_7", "5_8", "5_9", "6_7", "6_8", 
"6_9", "7_8", "7_9", "8_9", "0_0", 
"1_1", "2_2", "3_3", "4_4", "5_5", 
"6_6", "7_7", "8_8", "9_9", "pos_1_0_freq", 
"pos_1_1_freq", "pos_1_2_freq", "pos_1_3_freq", "pos_1_4_freq", "pos_1_5_freq", 
"pos_1_6_freq", "pos_1_7_freq", "pos_1_8_freq", "pos_1_9_freq", "pos_2_0_freq", 
"pos_2_1_freq", "pos_2_2_freq", "pos_2_3_freq", "pos_2_4_freq", "pos_2_5_freq", 
"pos_2_6_freq"

In [17]:
numeric_features = ["DrawNo",
                    "DaysSinceLastExactMatch", "DaysUntilNextExactMatch", 
                    "DaysSinceLastAnyMatch", "DaysUntilNextAnyMatch",
                    "DrawYear", "DrawMonth", "DrawWeek", "DrawDay", "DrawDayofweek", "DrawDayofyear",
                    "CumProbaExactMatch", "CumProbaAnyMatch",
                    "0_1", "0_2", "0_3", "0_4", "0_5", 
                    "0_6", "0_7", "0_8", "0_9", "1_2", "1_3", "1_4", "1_5", 
                    "1_6", "1_7", "1_8", "1_9", "2_3", "2_4", "2_5", "2_6", 
                    "2_7", "2_8", "2_9", "3_4", "3_5", "3_6", "3_7", "3_8", 
                    "3_9", "4_5", "4_6", "4_7", "4_8", "4_9", "5_6", "5_7", 
                    "5_8", "5_9", "6_7", "6_8", "6_9", "7_8", "7_9", "8_9", 
                    "0_0", "1_1", "2_2", "3_3", "4_4", "5_5", "6_6", "7_7", 
                    "8_8", "9_9",
                    "pos_1_0_freq", "pos_1_1_freq", "pos_1_2_freq", "pos_1_3_freq", "pos_1_4_freq", "pos_1_5_freq", 
                    "pos_1_6_freq", "pos_1_7_freq", "pos_1_8_freq", "pos_1_9_freq", "pos_2_0_freq", "pos_2_1_freq", "pos_2_2_freq", "pos_2_3_freq", 
                    "pos_2_4_freq", "pos_2_5_freq", "pos_2_6_freq", "pos_2_7_freq", "pos_2_8_freq", "pos_2_9_freq", "pos_3_0_freq", "pos_3_1_freq", 
                    "pos_3_2_freq", "pos_3_3_freq", "pos_3_4_freq", "pos_3_5_freq", "pos_3_6_freq", "pos_3_7_freq", "pos_3_8_freq", "pos_3_9_freq", 
                    "pos_4_0_freq", "pos_4_1_freq", "pos_4_2_freq", "pos_4_3_freq", "pos_4_4_freq", "pos_4_5_freq", "pos_4_6_freq", "pos_4_7_freq", 
                    "pos_4_8_freq", "pos_4_9_freq"
                   ]
categorical_features = ["PrizeType"]
datetime_features = ["DrawDate"]
target_feature = ["LuckyNo"]
len(numeric_features)

108

### Scaling

In [23]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
nf_transformed = scaler.fit_transform(df[numeric_features])
print(scaler.inverse_transform(nf_transformed))

/Users/mengwangk/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)


[[4.07920e+04 0.00000e+00 2.80000e+01 ... 6.00000e+00 2.00000e+00
  6.00000e+00]
 [4.07920e+04 0.00000e+00 2.43600e+03 ... 6.00000e+00 2.00000e+00
  6.00000e+00]
 [4.07920e+04 0.00000e+00 2.50000e+01 ... 6.00000e+00 2.00000e+00
  6.00000e+00]
 ...
 [4.95819e+05 1.20000e+02 0.00000e+00 ... 1.03720e+04 1.04320e+04
  1.02760e+04]
 [4.95819e+05 5.92000e+02 0.00000e+00 ... 1.03720e+04 1.04320e+04
  1.02760e+04]
 [4.95819e+05 2.06600e+03 0.00000e+00 ... 1.03720e+04 1.04320e+04
  1.02760e+04]]


/Users/mengwangk/anaconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [26]:
from sklearn.preprocessing import OneHotEncoder
onehot_encoder = OneHotEncoder()

onehot_encoder?

Type:        OneHotEncoder
String form:
OneHotEncoder(categorical_features=None, categories=None,
       dtype=<class 'numpy.float64'>, handle_unknown='error',
       n_values=None, sparse=True)
File:        ~/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py
Docstring:  
Encode categorical integer features as a one-hot numeric array.

The input to this transformer should be an array-like of integers or
strings, denoting the values taken on by categorical (discrete) features.
The features are encoded using a one-hot (aka 'one-of-K' or 'dummy')
encoding scheme. This creates a binary column for each category and
returns a sparse matrix or dense array.

By default, the encoder derives the categories based on the unique values
in each feature. Alternatively, you can also specify the `categories`
manually.
The OneHotEncoder previously assumed that the input features take on
values in the range [0, max(values)). This behaviour is deprecated.

This encoding is needed fo

In [14]:
corr_matrix = df.corr()
print(corr_matrix['LuckyNo'].sort_values(ascending=False))

LuckyNo                    1.000000
1st_digit                  0.994971
2nd_digit                  0.098613
3rd_digit                  0.010338
DaysUntilNextExactMatch    0.005985
DaysSinceLastExactMatch    0.005985
DrawDayofweek              0.004380
DaysUntilNextAnyMatch      0.003307
DaysSinceLastAnyMatch      0.002328
4th_digit                  0.001711
DrawDay                    0.000759
DrawWeek                  -0.002762
DrawDayofyear             -0.002896
DrawMonth                 -0.002985
DrawYear                  -0.003071
pos_2_8_freq              -0.003161
8_9                       -0.003164
pos_3_8_freq              -0.003164
pos_4_8_freq              -0.003165
pos_1_8_freq              -0.003165
0_8                       -0.003177
pos_2_9_freq              -0.003180
pos_4_9_freq              -0.003181
1_8                       -0.003181
pos_1_9_freq              -0.003184
pos_3_9_freq              -0.003185
3_6                       -0.003188
7_8                       -0

"DrawNo", 
"DrawDate", "PrizeType", "LuckyNo", "DaysSinceLastExactMatch", "DaysUntilNextExactMatch", "DaysSinceLastAnyMatch", "DaysUntilNextAnyMatch", "1st_digit", 
"2nd_digit", "3rd_digit", "4th_digit", "DrawYear", "DrawMonth", "DrawWeek", "DrawDay", "DrawDayofweek", 
"DrawDayofyear", "CumProbaExactMatch", "CumProbaAnyMatch", "0_1", "0_2", "0_3", "0_4", "0_5", 
"0_6", "0_7", "0_8", "0_9", "1_2", "1_3", "1_4", "1_5", 
"1_6", "1_7", "1_8", "1_9", "2_3", "2_4", "2_5", "2_6", 
"2_7", "2_8", "2_9", "3_4", "3_5", "3_6", "3_7", "3_8", 
"3_9", "4_5", "4_6", "4_7", "4_8", "4_9", "5_6", "5_7", 
"5_8", "5_9", "6_7", "6_8", "6_9", "7_8", "7_9", "8_9", 
"0_0", "1_1", "2_2", "3_3", "4_4", "5_5", "6_6", "7_7", 
"8_8", "9_9", "pos_1_0_freq", "pos_1_1_freq", "pos_1_2_freq", "pos_1_3_freq", "pos_1_4_freq", "pos_1_5_freq", 
"pos_1_6_freq", "pos_1_7_freq", "pos_1_8_freq", "pos_1_9_freq", "pos_2_0_freq", "pos_2_1_freq", "pos_2_2_freq", "pos_2_3_freq", 
"pos_2_4_freq", "pos_2_5_freq", "pos_2_6_freq", "pos_

In [6]:
columns_to_drop = [ "DaysSinceLastExactMatch", "DaysUntilNextAnyMatch",
                    "1st_digit", "2nd_digit", "3rd_digit", "4th_digit",
                    "0_1", "0_2", "0_3", 
                    "0_4", "0_5", "0_6", "0_7", "0_8", "0_9", "1_2", "1_3", 
                    "1_4", "1_5", "1_6", "1_7", "1_8", "1_9", "2_3", "2_4", 
                    "2_5", "2_6", "2_7", "2_8", "2_9", "3_4", "3_5", "3_6", 
                    "3_7", "3_8", "3_9", "4_5", "4_6", "4_7", "4_8", "4_9", 
                    "5_6", "5_7", "5_8", "5_9", "6_7", "6_8", "6_9", "7_8", 
                    "7_9", "8_9", "0_0", "1_1", "2_2", "3_3", "4_4", "5_5", 
                    "6_6", "7_7", "8_8", "9_9",
                    "pos_1_0_freq", "pos_1_1_freq", "pos_1_2_freq", "pos_1_3_freq", 
                    "pos_1_4_freq", "pos_1_5_freq", "pos_1_6_freq", "pos_1_7_freq", "pos_1_8_freq", "pos_1_9_freq", "pos_2_0_freq", "pos_2_1_freq", 
                    "pos_2_2_freq", "pos_2_3_freq", "pos_2_4_freq", "pos_2_5_freq", "pos_2_6_freq", "pos_2_7_freq", "pos_2_8_freq", "pos_2_9_freq", 
                    "pos_3_0_freq", "pos_3_1_freq", "pos_3_2_freq", "pos_3_3_freq", "pos_3_4_freq", "pos_3_5_freq", "pos_3_6_freq", "pos_3_7_freq", 
                    "pos_3_8_freq", "pos_3_9_freq", "pos_4_0_freq", "pos_4_1_freq", "pos_4_2_freq", "pos_4_3_freq", "pos_4_4_freq", "pos_4_5_freq", 
                    "pos_4_6_freq", "pos_4_7_freq", "pos_4_8_freq", "pos_4_9_freq"
                  ]
df.drop(columns_to_drop,inplace=True, axis=1, errors='ignore')
df.columns

Index(['DrawNo', 'DrawDate', 'PrizeType', 'LuckyNo', 'DaysUntilNextExactMatch',
       'DaysSinceLastAnyMatch', 'DrawYear', 'DrawMonth', 'DrawWeek', 'DrawDay',
       'DrawDayofweek', 'DrawDayofyear', 'CumProbaExactMatch',
       'CumProbaAnyMatch'],
      dtype='object')

In [7]:
df.describe()

,DrawNo,LuckyNo,DaysUntilNextExactMatch,DaysSinceLastAnyMatch,DrawYear,DrawMonth,DrawWeek,DrawDay,DrawDayofweek,DrawDayofyear,CumProbaExactMatch,CumProbaAnyMatch
count,104673.000000,104673.000000,104673.000000,104673.000000,104673.000000,104673.000000,104673.000000,104673.000000,104673.000000,104673.000000,104673.000000,104673.000000
mean,268299.492419,4974.001777,764.409447,66.179053,2005.754779,6.501648,26.452428,15.628214,4.054054,182.380356,0.062382,0.930387
std,131388.886981,2885.324727,832.538166,93.325465,7.809743,3.462904,15.123744,8.942459,1.843042,105.868193,0.038001,0.675621
min,40792.000000,0.000000,0.000000,0.000000,1992.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.010000,0.010000
25%,154499.000000,2463.000000,157.000000,15.000000,1999.000000,3.000000,13.000000,8.000000,2.000000,90.000000,0.030000,0.380000
50%,268306.000000,4952.000000,504.000000,39.000000,2006.000000,7.000000,26.000000,16.000000,5.000000,182.000000,0.060000,0.800000
75%,382113.000000,7463.000000,1095.000000,83.000000,2013.000000,10.000000,40.000000,24.000000,6.000000,274.000000,0.090000,1.310000
max,495819.000000,9999.000000,8087.000000,4113.000000,2019.000000,12.000000,53.000000,31.000000,6.000000,366.000000,0.270000,2.930000
